In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("smoking.csv")
df.head()

,age,gender,A1,A2_completed_education,A3,A4,A5,A6,A7,A8,...,Low,Middle,High,Not_applicable,Smoke_daily,Smoke_less_then_daily,Smoked_in_the_past,Never_smoker,a62_income,a4_smoking_status
0,35,2,1,2,2,1,NaN,2.0,3.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,34,1,4,3,3,1,NaN,2.0,3.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,37,2,3,3,3,1,NaN,4.0,2.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0
3,37,2,3,3,3,1,NaN,4.0,2.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0
4,35,2,2,3,3,3,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,3.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588 entries, 0 to 1587
Columns: 119 entries, age to a4_smoking_status
dtypes: float64(49), int64(70)
memory usage: 1.4 MB


## Data Cleaning

In [5]:
target = 'healthstatus3'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('#NULL!',np.nan,regex=True)
df = df.replace('99',np.nan,regex=True)
df = df.replace(99,np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = []
df = df.drop(outliers,axis=1)
df.head()

,age,gender,A1,A2_completed_education,A3,A4,A5,A6,A7,A8,...,Low,Middle,High,Not_applicable,Smoke_daily,Smoke_less_then_daily,Smoked_in_the_past,Never_smoker,a62_income,a4_smoking_status
0,35,2,1,2,2,1,NaN,2.0,3.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,34,1,4,3,3,1,NaN,2.0,3.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,37,2,3,3,3,1,NaN,4.0,2.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0
3,37,2,3,3,3,1,NaN,4.0,2.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0
4,35,2,2,3,3,3,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,3.0


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,200)
df.info()

number of bad columns: 23
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588 entries, 0 to 1587
Data columns (total 96 columns):
age                             1588 non-null int64
gender                          1588 non-null int64
A1                              1588 non-null int64
A2_completed_education          1588 non-null int64
A3                              1588 non-null int64
A4                              1588 non-null int64
A9                              1571 non-null float64
A16                             1587 non-null float64
A17                             1587 non-null float64
A18                             1588 non-null int64
A19                             1588 non-null int64
A20                             1588 non-null int64
A21                             1556 non-null float64
A22                             1585 non-null float64
A23                             1588 non-null int64
A24                             1588 non-null int64
A25                      

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,10)
df.info()

number of bad rows: 14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588 entries, 0 to 1587
Data columns (total 96 columns):
age                             1588 non-null int64
gender                          1588 non-null int64
A1                              1588 non-null int64
A2_completed_education          1588 non-null int64
A3                              1588 non-null int64
A4                              1588 non-null int64
A9                              1571 non-null float64
A16                             1587 non-null float64
A17                             1587 non-null float64
A18                             1588 non-null int64
A19                             1588 non-null int64
A20                             1588 non-null int64
A21                             1556 non-null float64
A22                             1585 non-null float64
A23                             1588 non-null int64
A24                             1588 non-null int64
A25                         

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 14


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  58    2    5    3    5    4    5    2    2    2    2    2    2    3
    3    3    3    5    2    2    2    3    2    2    2    2    2    2
    7    8    2    2    2    2    2    2    2    3    3    3    3    2
 1565   10    3    2    2    2    5    2    3    7    3    2    2    2
    2    2    2    2    2    2    4    2    2    2    2    2    2    2
    2    9    2    9    2    2    2    2    2    2    2    2    2    2
    2    2    2    2    2    2    2    2    2    3    4]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [1, 7, 8, 9, 10, 11, 12, 18, 19, 20, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 41, 45, 46, 47, 49, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
i_category: [3, 5, 13, 14, 15, 16, 21, 37, 38, 39, 40, 44, 50, 52, 62, 93, 94]
variable type: [3. 1. 3. 2. 3. 2. 3. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 3. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 1. 3. 3. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 1. 3. 3. 2. 1. 1. 1.
 3. 1. 2. 3. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 3.
 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1574 entries, 0 to 1573
Data columns (total 95 columns):
age                             1574 non-null int64
gender                          1574 non-null int64
A1                              1574 non-null int64
A2_completed_education          1574 non-null int64
A3                              1574 non-null int64
A4                              1574 non-null int64
A9                              1574 non-null float64
A16                             1574 non-null float64
A17                             1574 non-null float64
A18                             1574 non-null int64
A19                             1574 non-null int64
A20                             1574 non-null int64
A21                             1574 non-null float64
A22                             1574 non-null float64
A23                             1574 non-null int64
A24                             1574 non-null int64
A25                             1574 non-null int64

## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(1574, 132)
[[35.  1.  1. ...  0.  0.  0.]
 [34. -1.  4. ...  0.  0.  0.]
 [37.  1.  3. ...  0.  0.  0.]
 ...
 [28. -1.  4. ...  0.  0.  1.]
 [29. -1.  2. ...  1.  0.  0.]
 [47.  1.  1. ...  0.  0.  0.]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([852, 722]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')